# Crawling Web Berita CNN Indonesia

## Apa itu Crawling?

Crawling merupakan proses search engine untuk menemukan konten atau sesuatu situs halaman yang ada. Dalam bahasa kerennya crawling atau web crawling merupakan proses dimana search engine mengirimkan bot atau robot yang disebut (crawler atau spider) yang digunakan untuk menemukan konten-konten yang ada.

Yang dimaksud konten yaitu bervariasi, mulai dari halaman website yang saya lakukan ini, kemudian gambar, video, dokumen, dan lain sebagainya. Seperti halnya laba-laba, datang ke sebuah jaring dan melihat beberapa halaman website, kemudian mengikuti link yang terdapat di halaman website tersebut untuk mencari URL yang baru.

Ketika ada pengguna yang mencari sebuah konten di search engine dengan keyword tertentu, search engine akan mencarinya di indeks dan menentukan konten mana yang paling sesuai untuk pengguna tersebut.

## Proses Crawling

### Tool atau Library

In [385]:
# Import Library yang dibutuhkan
import requests as req
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from tqdm import tqdm

* **Request** digunakan untuk mengambil html/http dari sebuah website.
* **BeautifulSoup** berfungsi untuk mengambil data dari html/xml.
* **Time** berfungsi untuk memberikan jeda ketika ingin berpindah halaman.
* **Pandas** digunakan untuk membuat dataframe agar mudah dibaca.
* **tqdm** Untuk mentracking proses program

### Code Program

#### Fungsi Clean_text()

In [386]:
def clean_text(text):
	text = text.replace('\xa0', '',)
	return text.strip()

Fungsi untuk membersihkan text yang tidak diinginkan, atau mengganggu.

#### Fungsi scrape_news()

In [387]:
def scrape_news(soup):
	berita = {}
	texts = []
	# TODO:
	# ada struktur aneh https://www.cnnindonesia.com/olahraga/20240830134615-142-1139388/live-report-timnas-indonesia-vs-thailand-u-20
	
	berita["judul"] = soup.title.text

	text_list = soup.find("div", class_="detail-text text-cnn_black text-sm grow min-w-0")
	for text in text_list.find_all("p"):
		if 'para_caption' not in text.get('class', []):
			cleaned_text = clean_text(text.text)
			texts.append(cleaned_text)
		
	berita["isi"] = "\n".join(texts)
	berita["tanggal"] = soup.find("div", class_="container !w-[1100px] overscroll-none").find_all("div")[1].find_all("div")[4].text
	berita["kategori"] = soup.find("a", attrs={"aria-label": "link description", "dtr-act": "kanal"}).text
	return berita

Pada fungsi ini berisikan proses pembedahan dan juga pengambilan data pada sebuah website. Mengambil data sesuai struktur HTML/web yang ingin diambil datanya.

#### Fungsi get_html()

In [388]:
def get_html(url):
	try:
		response = req.get(url).text
		return bs(response, "html5lib")
	
	except Exception as e:
		print(e)
		return ""

Fungsi get_html dengan parameter url digunakan untuk mengambil response atau isi html dari web. Untuk mengambil response tersebut dibutuhkan library request, dan juga BeautifulSoup untuk mendapatkan isi html.

#### Fungsi get_news()

In [389]:
def get_news(soup):
	container = soup.find("div", class_="container !w-[1100px] overscroll-none")
	news_list = container.find_all("article", class_="flex-grow")
	return news_list

Fungsi get_news berfungsi untuk mengambil semua berita yang ada pada web, yang kemudian didapat kumpulan url berita yang ada pada halaman web.

#### Main Crawling

In [390]:
def main(halaman=1):
	url = ["https://www.cnnindonesia.com/nasional/indeks/3/", "https://www.cnnindonesia.com/internasional/indeks/6/"]
	# url = ["https://www.cnnindonesia.com/internasional/indeks/6/"]
	news = []
	count = 1 # - n

	for _ in range(len(url)):
		url_now = url.pop()
		for counter in range(halaman):
			page = url_now + str(count)    
			soup = get_html(page)
			news_list = get_news(soup)
			
			for item in tqdm(news_list, desc=f"Processing page {count}"):
				news_url = item.find('a')['href']
				# print(news_url)
				soup_news = get_html(news_url)
				result = scrape_news(soup_news)
				news.append(result)
			count+=1
			time.sleep(1)
		
	return news

Menyiapkan link/base url web berita yang ingin dicrawling, terdapat beberapa fungsi yang dipanggil yang sudah dibuat sebelumnya untuk mengambil informasi atau berita pada halaman website. Dalam code tersebut terdapat beberapa tahapan seperti fungsi:
* get_html
* get_news
* scrape_news

#### Main

In [391]:
# halaman = int(input('Crawling berapa Halaman? '))
halaman = 5
news = main(halaman)

Processing page 10: 100%|██████████| 10/10 [00:04<00:00,  2.15it/s]


Menjalankan program yang sudah dibuat dengan input berapa halaman yang ingin diambil.

### Convert Dataframe

In [394]:
df = pd.DataFrame.from_dict(news)
df
df.to_csv("data_100.csv", index=False)

In [393]:
print(df['isi'][0])

Presiden Rusia Vladimir Putinbergurau bahwa dirinya mendukung calon Presiden (capres) Amerika Serikat dari Partai Demokrat, Kamala Harris, alih-alih capres dari Partai Republik, Donald Trump.
Putin berkelakar Harris memiliki tawa yang menular yang mencerminkan bahwa dunia baik-baik saja.

"Dia tertawa begitu ekspresif dan menular sehingga itu berarti semuanya baik-baik saja dengannya," kata Putin, seperti dikutip Reuters.
Putin melontarkan candaan ini saat sedang menjawab pertanyaan moderator di acara forum ekonomi di Rusia pada Kamis (5/9). Sang moderator kala itu bertanya bagaimana pandangan Putin mengenai pemilihan umum di AS.
Putin mulanya menjawab serius bahwa pemilihan umum di AS tergantung pada pilihan rakyat Amerika.
Namun, dia berkelakar bahwa Presiden petahana Joe Biden saat ini mendukung Harris. Oleh sebab itu, dirinya pun akan memilih Harris.
"Kami akan melakukan hal yang sama, kami akan mendukungnya," kata Putin.






Awal tahun ini, Putin sempat berkomentar bahwa dirinya